# RSA演算法

[米勒-拉賓質數判定法](https://zh.wikipedia.org/wiki/%E7%B1%B3%E5%8B%92-%E6%8B%89%E5%AE%BE%E6%A3%80%E9%AA%8C)

In [1]:
import base64
import random


class RSA():
    def __init__(self, bits: int, e: int = 65537):
        self.e = e
        self.bits = bits
        self.__rsa(bits, e)
        
    def __rsa(self, bits, e):
        for _ in range(2):
            while True:
                prime_bits = '1' + ''.join([random.choice(['0', '1']) for __ in range(bits//2-2)]) + '1'
                prime = int(prime_bits, 2)
                if self.__miller_rabin(prime, 3):
                    if _ == 0:
                        self.p = prime
                    else:
                        self.q = prime
                    break
                else:
                    continue
        self.n = self.p * self.q
        phi_n = (self.p - 1) * (self.q - 1)
        self.d = self.__euclid(self.e, phi_n)

    def __miller_rabin(self, n, k):
        if n == 2 or n == 3:
            return True

        if n % 2 == 0:
            return False

        r, s = 0, n - 1
        while s % 2 == 0:
            r += 1
            s //= 2

        for _ in range(k):
            a = random.randrange(2, n - 1)
            x = pow(a, s, n)
            if x == 1 or x == n - 1:
                continue
            for _ in range(r - 1):
                x = pow(x, 2, n)
                if x == n - 1:
                    break
            else:
                return False
        return True
    
    def __euclid(self, e: int, n: int) -> int:
        r1 = n
        r2 = e
        t1 = 0
        t2 = 1
        while r1 != 1:
            q = r1 // r2
            r = r1 % r2
            r1 = r2
            r2 = r
            t = t1 - (q * t2)
            t1 = t2
            t2 = t
        if t1 < 0:
            t1 += n
        return t1
    
    def exponentiation(self, base, exp, N):
        t = 1;
        while(exp > 0): 

            # for cases where exponent
            # is not an even value
            if (exp % 2 != 0):
                t = (t * base) % N;

            base = (base * base) % N;
            exp = exp // 2;
        return t % N;
        
    def encrypt(self, plain_text: str) -> tuple:
        cipher = []
        for char in plain_text:
            cipher.append(base64.b64encode(self.exponentiation(int.from_bytes(char.encode('big5'), 'little'), self.e, self.n).to_bytes(self.bits//8, byteorder='little')).decode('utf-8'))
        return tuple(cipher)
            
    def decrypt(self, cipher: tuple) -> tuple:
        msg = []
        for char in cipher:
            msg.append((self.exponentiation(int.from_bytes(base64.b64decode(char.encode('utf-8')), 'little'), self.d, self.n)).to_bytes(2, 'little').decode('big5'))
        return tuple(msg)

In [2]:
rsa_key = RSA(2048)

In [3]:
print(f'\n私鑰：\033[33m{rsa_key.d}\033[0m\n\n公鑰：\033[33m{rsa_key.e}\033[0m')


私鑰：9725877513197068606908179978786088553016024622197782171727238799609448429311739343397412840307099682629916248369021924134354608493146373914445894984406323097175518209188061101879404061621607245727194367861988035088774796037829913318710793446640917114275798091505234119255577026820055652793723058008035505614665984833331698671508440780847567140536368123949424355971237477695242543362719480399482207904606220169892302853594463062183959665653424500209444881169833212900476536314372721103962402931763792645795130010222384775428885690628307662492363119782617178087447970861517705651772750826879769929559732778965455200353

公鑰：65537


In [4]:
plain_text = "All I ask is if this is my last night with you, hold me like I'm more than just a friend. Give me a memory I can use."

In [5]:
cipher = rsa_key.encrypt(plain_text)
print(''.join(cipher))

X6upbchuMH4hFjMd8DaFw3bzyMoGGORsuhAwmkB/C7WbB1nBlQtS178u5UnZ0FnSV56vJpHpbP6WWBqINBJvk16du0EdcnzbPSaGQ0FdpwmVGcc2Vscu5XgeXGNI7+fQg7G73V/tEwrnMDV04+4IH2PYv7ACrZk3F5+TrbkOUqrW8J5o+xOO8gjuLMJGePH6K6OcIRHcjcnHBuLUG7u8CBKJetOKqYQtIjZ8xIEsjLm+K5dZSs2KHddeS92M6KPcdBDqq5d8Nuj2+0LAs7hLuenrWk3Q/IkVThlYrOc8bu/EwBr3YZW/+ch91blIKOcv1A7+fH2gKLfXC+ojC1HyFQ==1/9uiNfRSGxExBETw9Y2A0sNkjZ/ivBk7x9O0Em8sb6qD3zMvnWYpskdGmXmZ+91s9mRsFxXUNc2QxOkhdNc9pXOp35aJTbWQ+2KI88LWw6FY5gDU0znTp8xBQy1DafKdHZckq7fE7H+f0Sss4TQHTPYWfRnDIg6Fhcfa7KIkqVlzGr9S2klj5s/KcC2HTnKY0qiJFKWJZuu9c2YSEGHVhoMKtIJ3/QoAl3gHVgLxTWYzV3yc24XqqRGpChgmMOgS6wVn8bjyRR6Lsz41HQmwdHqiDPZBf+FD+ghnt05QiqQJQrb+z+9Sc+DI2xsGpuXA6w+QxL7/WWCeMy0GT+Ygg==1/9uiNfRSGxExBETw9Y2A0sNkjZ/ivBk7x9O0Em8sb6qD3zMvnWYpskdGmXmZ+91s9mRsFxXUNc2QxOkhdNc9pXOp35aJTbWQ+2KI88LWw6FY5gDU0znTp8xBQy1DafKdHZckq7fE7H+f0Sss4TQHTPYWfRnDIg6Fhcfa7KIkqVlzGr9S2klj5s/KcC2HTnKY0qiJFKWJZuu9c2YSEGHVhoMKtIJ3/QoAl3gHVgLxTWYzV3yc24XqqRGpChgmMOgS6wVn8bjyRR6Lsz41HQmwdHqiDPZBf+FD+ghnt05QiqQJQrb+z+9Sc+D

In [6]:
''.join(rsa_key.decrypt(cipher)).replace('\x00', '')

"All I ask is if this is my last night with you, hold me like I'm more than just a friend. Give me a memory I can use."

---

# 數位信封

## 產生對稱式金鑰

### 沒有 IV 會導致前 16 個字無法解碼

In [7]:
def generate_aes_key(bytes: int) -> str:
    key = b''.join([(random.randint(0, 0xFF).to_bytes(1, 'little')) for _ in range(bytes)])
    return key

aes_key = generate_aes_key(32)
IV = generate_aes_key(16)
print(aes_key, IV, sep='\n')

b'\x1b\x90g\xdc\\\x9c`sa\xbcz\x1a\x13\xcb\xef\xc9h\x9f~C\xa58\x0e\xcb\x8a&\x8a,,T\x1e1'
b'of\xbeH\x9a\xfeO\x8e\xa0\xbeN\x83\xfd\xda\x8fL'


## 加密明文

In [8]:
plain_text = "All I ask is if this is my last night with you, hold me like I'm more than just a friend. Give me a memory I can use."

### Padding

In [9]:
def padding(text):
    if type(text) is str:
        text = text.encode('utf8')
    elif type(text) is bytes:
        pass
    else:
        raise TypeError('請輸入字串或位元組資料')
    if len(text) % 16 != 0:
        text += ('\0'.encode('utf8') * (16 - (len(text) % 16)))
        
    return text

In [10]:
plain_text = padding(plain_text)
plain_text

b"All I ask is if this is my last night with you, hold me like I'm more than just a friend. Give me a memory I can use.\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"

In [11]:
len(plain_text)

128

In [12]:
from Crypto.Cipher import AES

cipher_aes = AES.new(aes_key, AES.MODE_CBC, IV)
try:
    cipher = cipher_aes.encrypt(plain_text)
except TypeError:
    cipher = cipher_aes.encrypt(plain_text.encode())
except Exception as e:
    print(e)

In [13]:
cipher

b'\xc3\xfe-\x00\xf1M#\xe9\xc5e\xd9\x98\x8f\x0fr\x8e\x98\xdb\xe6\x92\x83H\x94\x02\x7f\t\x85\xa4&`\xe0\xfa r[U\xe7\x03\x1dJy"5\x07\x01\xf9u\x9f\x8e\xb3/B\x89\x98\xcf*\x9bO\x06\xd8\xd4QD\tC\xbc+\xb9\x13Pi\xeeX\xaa\xceM\x9c\xb1F+Q\xec\x17zGp\xd4\xb7\xf5\xe6DC\xd9p\xc73\xf2\xab\xab*\xa2\xbfE);\x9e\xa2\xa0e\xc9\x87\x16u?\xc2\xc5\x19\xaa\xa40^U\xceS\xeaZy\xd2'

## 數位信封RSA演算法

In [14]:
class DigitalRSA(RSA):  
    def encrypt(self, secret_key: bytes):
        cipher = []
        for B in secret_key:
            cipher.append(base64.b64encode(self.exponentiation(B, self.e, self.n).to_bytes(self.bits//8, byteorder='little')))
        return cipher
    
    def decrypt(self, encrypted_secret_key: list):
        decrypted_key = []
        for B in encrypted_secret_key:
            decrypted_key.append(self.exponentiation(int.from_bytes(base64.b64decode(B), 'little'), self.d, self.n).to_bytes(1, 'little'))
        return decrypted_key

## 加密AES金鑰

In [15]:
d_rsa = DigitalRSA(2048)
key = d_rsa.encrypt(aes_key)

In [16]:
print(f'\n私鑰：\033[33m{d_rsa.d}\033[0m\n\n公鑰：\033[33m{d_rsa.e}\033[0m]')


私鑰：5356437298160291059319730104423405030154238107520981469014804225150237671135167626192977617089664028413833876615658227561948570375600214507010363061149015524059751443236903343657421453954435678661844903568723343898163674674379423872320993975373634485197260258358006160800406812203526205108147354975444653935216006872257794344345459320990601215545593321767085440248265222245031127420348684482211216972512962877957004024486251520168959711720273411986389621869835634015907494683226000239396491252149149555869095852081422756270698573659430211363076857011850305336914792568732115128043938732836117438575799002459818439553

公鑰：65537]


In [17]:
b''.join(key)

b'DZpj1fVHElGBcNnGVP25hiF4y8Tc7cT8os2Vfnoyvl8giqtHE4fj4kcgOz+ITCfRfV3DQGFkgcjj+aRQS/mWyG+8k7XKHwbMrskqKZiPy3FeoOjNA3Z86VhUZ9JdoUDKpRe1CmDur0ZkS9mMYGl4lftG70AJjkcmP8ZWy3g49QPqUfIhIVzwTEflLsi9mOiNDFjFhI5RTcB60N9Bky5q3jBtELYp+dhhyNVpyvyF62c8kNdJWhwttXdQoE4JNURpe9H9J4vI8DB2Lmvev5mNNpJiADCaEwAAuTWdS0qJEJNw0gVTOFJzdylFsNhaKnAOrb05BMtY3rdS15+ony5Ofw==Iq/qrxsvf99f4urgkY9VNiGfML/nXI/wtWoHmlP3NnZ7VyGU0QZKViobmPndZmJoE1g1pzOjrb5S/THbtNY7JoqkYc+seVcn3FEEGwfDqgmrmGwEiR/c8WwV6X68zr80NSCu/YFpmuKdVxKSOig5XPuyNm2duLYjlVQoCP0ZIvqV206uQ4lMndIgN1n/MXjbGPHGrKUVAYkXp90EGP1V+OJUIhcJ7vUPEVJ/f4COZ068/+UOi5qpZSnP8eCaph0qPamHJ6AjVsHgCWn9mBtBTds0tSMWJkmZ4XFea+MRhZLg8HyXaYHVCbEfwVmp7llnxGrFVeNzv3gx31pPDQaZaw==htTlC3ThRpDjcxtu77p9zI8zKlIylLUCeM2eLPPHyGB80tJ1g1WuU3po2rxUHXpJ1SRKmyIaQJw6gGuERCGj7fWiIL1rr0xuw5RSlxYHUk+CQKYbcCbL9OLp/+DzBW69R66iIo5V/Pbs6PfPYM9pRVwSoAmBYMAcoW1kxYfR+bmGICPkOFhgOrcU5V78PlG8SBPej1T1y0+t+AJcezT2TBd36UQ5hd/hL4KT9AJeAIzK65rYLrlYO4qyA2dXXHEbO/sSanl/FYmz721jBVjA1xGw/QlO/LZYxk0xSYzhi9RHqOREG2LUQw

## 解密AES金鑰

In [18]:
s = bytes()
for _ in d_rsa.decrypt(key):
    s += _

In [19]:
print(f'{"Decrypted secret key:":>12} \033[33m{s}\033[0m\n{"Original secret key:":>{len("Decrypted secret key:")}} \033[33m{aes_key}\033[0m\n\nEqual? {s == aes_key}')

Decrypted secret key: b'\x1b\x90g\xdc\\\x9c`sa\xbcz\x1a\x13\xcb\xef\xc9h\x9f~C\xa58\x0e\xcb\x8a&\x8a,,T\x1e1'
 Original secret key: b'\x1b\x90g\xdc\\\x9c`sa\xbcz\x1a\x13\xcb\xef\xc9h\x9f~C\xa58\x0e\xcb\x8a&\x8a,,T\x1e1'

Equal? True


## 使用解密之AES金鑰解密秘文

In [20]:
decryptor = AES.new(s, AES.MODE_CBC, IV)

decryptor.decrypt(cipher).decode('utf8').replace('\x00', '')

"All I ask is if this is my last night with you, hold me like I'm more than just a friend. Give me a memory I can use."

---

# 長文章數位信封

In [21]:
import requests as rq
from bs4 import BeautifulSoup as bs


res = rq.get('https://mrmad.com.tw/apple-redesign-macbook-air-2021-new-color')
soup = bs(res.text, 'lxml')

paragraph = []
for p in soup.find_all('p')[: -3]:
    p = p.text
    if p != '':
        paragraph.append(p)
        
paragraph = '\n'.join(paragraph)
print(paragraph)

蘋果爆料達人 Jon Prosser 在最近獨家洩密新一代M2款 MacBook Air 將採用「彩色」外觀設計，與蘋果春季發表會推出的 24吋 iMac 顏色相同，規格與外觀也會迎來4大方面改進。
 
Jon Prosser 過去就曾準確爆料 iMac 2021年款會加入多種新顏色，這次又從可靠來源得知，下一代 M2 款 MacBook Air 外觀預計也會加入多款顏色，顏色與 M1 款 iMac 完全相同，共會有7種配色，分別是藍色、 綠色、 粉紅色、 銀色、 黃色、 橙色、 紫色。
且 Jon Prosser 也從蘋果內部知情人士得知，當前 Apple 內部已經有藍色 MacBook Air 原型機。
至於 2021 MacBook Air 新配色會如同底下這幾張模擬圖，主要是採用淡色系，另還會加入 MagSafe 磁吸充電。
 
至於 Apple 打算替 iMac 和 MacBook Air 加入多款彩色選擇，最主要是替產品等級區隔，對於一般消費者而言，追求的並非是效能，而是外觀和便利性，推出多顏色更能提升用戶接受度，對於需要高效能專業級別的用戶，硬體規格需求明顯大於顏色，也是會有黑色或灰色 Pro 級別款式可供選擇。
從去年 10月蘋果發表會，蘋果推出首款多顏色 iPad Air 後，明確顯示要讓產品走向繽紛色，同時要以顏色來區隔等級。
如同 M1款 iMac 和 iPhone 普通與專業款差異，對於一般型消費市場，則是以多顏色來搶攻普通用戶目光和荷包，同時也有專業 Pro 級別款式，則是以原色或深色款來突顯產品的強勁性能與規格。
過去彭博社（Bloomberg）記者 Mark Gurman 也曾爆料，蘋果已經準備研發新一代更輕薄的 MacBook Air ，預計會在 2021 下半年推出，連同蘋果分析師郭明錤也曾指出，會有一款低階 MacBook 會搭載 mini-LED 螢幕。
如從目前多方消息來看，新款 MacBook Air 2021 預計採用多顏色、M2處理器、MagSafe 磁吸充電和厚度將變更薄為主要特色，至於 mini-LED 螢幕預計會是給 MacBook Pro 系列才會搭載。
更新：MacBook Air 2021機身曝光，搶先看6大外觀改進和發表時間


In [22]:
from Crypto.Cipher import AES

class DigitalEnvelope():
    def __init__(self, key_bytes: int = 32, IV: bool = True):
        self.aes_key = self.__generate_aes_key(key_bytes)
        self.IV = self.__generate_aes_key(16) if IV else None
        print(f'{"AES secret key:":>12} \033[33m{self.aes_key}\033[0m\n{"IV:":>{len("AES secret key:")}} \033[33m{self.IV}\033[0m')
    
    def __generate_aes_key(self, key_bytes: int) -> bytes:
        key = b''.join([(random.randint(0, 0xFF).to_bytes(1, 'little')) for _ in range(key_bytes)])
        return key
    
    def __padding(self, text):
        if type(text) is str:
            text = text.encode('utf8')
        elif type(text) is bytes:
            pass
        else:
            raise TypeError('請輸入字串或位元組資料')
        if len(text) % 16 != 0:
            text += ('\0'.encode('utf8') * (16 - (len(text) % 16)))

        return text
    
    def encrypt(self, plain_text, secret_key, IV, mode = AES.MODE_CBC):
        plain_text = self.__padding(plain_text)
        cipher_aes = AES.new(secret_key, mode, IV)
        try:
            cipher = cipher_aes.encrypt(plain_text)
        except TypeError:
            cipher = cipher_aes.encrypt(plain_text.encode('utf8'))
        except AttributeError:
            cipher = cipher_aes.encrypt(plain_text)
        return cipher.decode('ISO-8859-1')

    def decrypt(self, cipher_text, secret_key, IV, mode = AES.MODE_CBC):
        decryptor = AES.new(secret_key, mode, IV)
        return decryptor.decrypt(cipher_text.encode('ISO-8859-1'))
            
            
class CipherKey(RSA):  
    def encrypt(self, secret_key: bytes) -> tuple:
        cipher_key = []
        for B in secret_key:
            cipher_key.append(base64.b64encode(self.exponentiation(B, self.e, self.n).to_bytes(self.bits//8, byteorder='little')))
        return tuple(cipher_key)
    
    def decrypt(self, encrypted_secret_key: tuple):
        decrypted_key = []
        for B in encrypted_secret_key:
            decrypted_key.append(self.exponentiation(int.from_bytes(base64.b64decode(B), 'little'), self.d, self.n).to_bytes(1, 'little'))
        s = bytes()
        for _ in decrypted_key:
            s += _
        return s

In [23]:
plain_text = paragraph
DE = DigitalEnvelope()
aes_key = DE.aes_key
IV = DE.IV

AES secret key: b"\xba\x18\x91~\x99\x02\x0eS+\x84|\xe1'\xd9\xdc\xd5:\x18:\xef<\xde\xcem8\xb6\xaa~\xa7\x0b\xa13"
            IV: b'\x94\xd4\xd6~\x94\xb6Mq8\xf0\xbf\xf4\x9ce\x17\x96'


In [24]:
cipher_text = DE.encrypt(plain_text, aes_key, IV)
cipher_text

'¼CJÀï4r\nÅ½ð\x17ýe\x8f@`a\'Îiºº\x11ØÐß\x8e\x0f7\x9fkåþGä\x9fÎ\r\x98a?\x03è:¤L®u\x858XN\x8e\x94\x1d¦qdOS·\x88ÿ\x80~Û?\x12\x99óñ\x9d\x82Û\x05M5Ì\x19Æ\x9cúç\x8c\x12îÂNßha*ê¥ÉOD\x05±_½à\x19§Ò\x98$]²\x1bqÂW,T¥\x96[î\x17(\x90-Á>\x07\x16\x8b\x02\x1bÖ\x8dÍb4«ù.G»\x88\x14\x0fÃÿ4º\x91\x97\t6#Xð\x1aïÂ\x83{6<V\x9f\n\x92ÒÐI\x12¡Üm\x1eÅeE\x84÷áuà¶\x81\x0e\x06t¤ágïQ¨\x87\x0b\x80;L\x89\x16\x9dÅÄå£\x13\x9fc¤\x85mÆSÔ)\x0e\x1dóÊ_§ôCO\x05êÂÜpLÓ~¨\x0f.4\x1aäÓ´´a\x9a\x92é·8×\x0c+\x17!4¤!8²ñûK\x89KN5|ï\x8a\x8déíú°S\x12öyR¯\x88ó0÷%®9K\x02U¿6Ðª\x7fåó;Lq¶ð\x03dû\x05ðJ¢=\x95\x99&_¯ßâj{ù\x80¼iã´8®\x97%TJ\x92CÕ\x96Ä_æ\x0cSÝpS´å¸Á\x1dKn\x1a\x8a\x12ÓSD\x8a\x1dÂ\x0bi^;o\x97TÑ\x10¯\rýâç)ÍÀ\x9dê^\x7f\x14A\x99~%G\xadÈ\xa0î¹ø¢\x93nfú¹k° BÂ\x86Î\x1bM¾[t\x944yM\x9f¯L\x7f\x86\x90Ea{C)ÜÒ\x98zù)\x17öô\x18¿äiHØý\x11\x0c¿á9\x06ÿ{Ñ\x92CÔÀO3£ûdVPà&|\x18:Ó\x10c\x07%$Ú^]¥¨iõ\x1e\x89vå!/>`¹²o\x9bùÒ¥5{+\x9bzÈ\x03WÌTWóú¼ÑìjN\x8c·IoCC\x14§zj+ë×(\x03)µ\x1ca\x19íúÔ¿A\x11v\x04¬¦i8:Ëd\x84pxR29,-ò¤ßa²lïºà\x07½1=9\x9e\röj~sÊl}µ\x94 T\x8dOìÆ

In [25]:
d_rsa = CipherKey(2048)
encrypted_aes_key = d_rsa.encrypt(aes_key)
b''.join(encrypted_aes_key)

b'A+qdFv0qnXV/ng0TX7T427/ekYSF0+trqWq2FzxSIRha9FNkQ6HLAbQTAg466c7wK5V34ZRdGxZaMzyXlqnV8Le6AewcWVj1IgbXL3ovhRb+G/93btLEj0VaHmeeP6AY6G0/1zI6tO8tIgpi8UEKDMtpnxIztzaIkyZ+HIma0MXLCGQG3SYrldTefA4gFkR6TtYf30vjL2D9284r18iBzf87W8G0dkG4x7zvomfd2sAvEajvRhqjcyzbwEIosSREuU9tLt4uATqDyHMufOm0H39Pqng0vBnIL/u7V8FHAmTKq3K9jg38rDASYaBLY/mc9erxTgMUYnzLDJ3xEcDHSQ==vB6YFcyu1M3WZ27gSyG9/LzdOB4xhAA2lfuMaY9ZInrt0siq8nrvWeI2aWU0Movh3TRltSMOEjCjULvJX1LIT4j+fnu0ve7FlUU8dhK1FICb7v453FHylgDJ8yGJepRMQqbqQGEJcxfJZfsrpthtqUbr1nb/hO/pSCwtZTOe6dz4Wy2FwFfsUpMHCHAbGZOPm9qEYyjm5zLySSi0g7/6e5hr4tB7Dn0bpfiDM98EN/7EgvUP8CFrjcIjHAm0W94tvJJXlFsGQwLQamRfLdbcfinZn3F6GB/DDSQFtcDFKEe23Tvsi4dzUzsBhjLpnTdp39nAk2bA+h2TUFc1pb68Aw==SkgDy3ruiobRgsMqdfU4zlhn8+P7WLWWrfZDu3nj0bx8KP9IrHhtQ7CgKrub4uqA3I1PcbC22enEwkFl9BwMI+gWH+FOhG//SciscZk/V/HpZYjNR4a4JzTOU2Jrk9rJ4fCL1B5Ar0nZGS4b+kvVP6tFlYeHb+FG8Q+WG2XrwESp1PqaX/kzUFMaW42F3edm/bIJ84xTiz8zf4S4j/RMjc0pPuV15QR1CNdKZn5Tx8s7gD3qaSCmm/lwcSAtGfeLNI+letXkER8ariKomrnVa9NOROUvVinosqAjIIQ8btmH5ianFzoQYm

In [26]:
decrypted_aes_key = d_rsa.decrypt(encrypted_aes_key)
decrypted_aes_key

b"\xba\x18\x91~\x99\x02\x0eS+\x84|\xe1'\xd9\xdc\xd5:\x18:\xef<\xde\xcem8\xb6\xaa~\xa7\x0b\xa13"

In [27]:
print(DE.decrypt(cipher_text, decrypted_aes_key, IV).decode('utf8').rstrip('\x00'))

蘋果爆料達人 Jon Prosser 在最近獨家洩密新一代M2款 MacBook Air 將採用「彩色」外觀設計，與蘋果春季發表會推出的 24吋 iMac 顏色相同，規格與外觀也會迎來4大方面改進。
 
Jon Prosser 過去就曾準確爆料 iMac 2021年款會加入多種新顏色，這次又從可靠來源得知，下一代 M2 款 MacBook Air 外觀預計也會加入多款顏色，顏色與 M1 款 iMac 完全相同，共會有7種配色，分別是藍色、 綠色、 粉紅色、 銀色、 黃色、 橙色、 紫色。
且 Jon Prosser 也從蘋果內部知情人士得知，當前 Apple 內部已經有藍色 MacBook Air 原型機。
至於 2021 MacBook Air 新配色會如同底下這幾張模擬圖，主要是採用淡色系，另還會加入 MagSafe 磁吸充電。
 
至於 Apple 打算替 iMac 和 MacBook Air 加入多款彩色選擇，最主要是替產品等級區隔，對於一般消費者而言，追求的並非是效能，而是外觀和便利性，推出多顏色更能提升用戶接受度，對於需要高效能專業級別的用戶，硬體規格需求明顯大於顏色，也是會有黑色或灰色 Pro 級別款式可供選擇。
從去年 10月蘋果發表會，蘋果推出首款多顏色 iPad Air 後，明確顯示要讓產品走向繽紛色，同時要以顏色來區隔等級。
如同 M1款 iMac 和 iPhone 普通與專業款差異，對於一般型消費市場，則是以多顏色來搶攻普通用戶目光和荷包，同時也有專業 Pro 級別款式，則是以原色或深色款來突顯產品的強勁性能與規格。
過去彭博社（Bloomberg）記者 Mark Gurman 也曾爆料，蘋果已經準備研發新一代更輕薄的 MacBook Air ，預計會在 2021 下半年推出，連同蘋果分析師郭明錤也曾指出，會有一款低階 MacBook 會搭載 mini-LED 螢幕。
如從目前多方消息來看，新款 MacBook Air 2021 預計採用多顏色、M2處理器、MagSafe 磁吸充電和厚度將變更薄為主要特色，至於 mini-LED 螢幕預計會是給 MacBook Pro 系列才會搭載。
更新：MacBook Air 2021機身曝光，搶先看6大外觀改進和發表時間
